## Import libs

In [2]:
#Import libs 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statistics as st
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
import random
import mlflow
import mlflow.sklearn
import geopandas as gpd
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 18)

## Upload main data

In [3]:
# # Loading the csv file
# path = 'C:\\jupyter\\SPP\\input\\'
# data_init = pd.read_csv(path + 'ACG_wells_JOINT_BEST_v7.csv', sep=',')
# # Data cleaning of TL-dataset
# df = data_init.copy()
# df = df[1:]
# #Select only neccessary data
# df_cln = df[['wellName', 'DEPTH', 'AREA', 'BADPORLOG', 'Casings', 'FORMATION',
#             'FLANK1', 'FLANK2', 'FLUIDS',
#             'LPERM', 'PHIT', 'NET', 
#             'GR_N', 'GRMATRIX', 'GRSHALE','VSH', 'NPSS', 'RHOB', 'RHOF', 'RHOMA', 
#             'RDEEP',  'SON', 'SONSH', 
#             'TVD_SCS','TST', 'DEVI','HAZI','X', 'Y', 'Dip_Azimuth', 'Dip_TRU']]
# #Fill up nan and -9999 values with 0
# df_cln = df_cln.fillna(0)
# df_cln = df_cln.replace(-9999, 0)
# df_cln = df_cln.replace('-9999', '0')
# #Assing proper datatypes for df
# dicttypes = {'wellName':'string', 'DEPTH':'float', 'AREA':'int', 'BADPORLOG':'int', 'Casings':'float', 'FLANK1':'int', 'FLANK2':'int',
#              'FLUIDS':'int','FORMATION':'string', 'GR_N':'float', 'GRMATRIX':'float', 
#              'GRSHALE':'float', 'LPERM':'float', 'NPSS':'float',
#              'PHIT':'float', 'NET':'float', 'RDEEP':'float', 'RHOB':'float', 'RHOF':'float', 'RHOMA':'float', 'TVD_SCS':'float', 'TST':'float',
#              'VSH':'float', 'X':'float', 'Y':'float', 'Dip_Azimuth':'float', 'Dip_TRU':'float'}
# df_cln = df_cln.astype(dicttypes, errors='ignore')
# df_cln.loc[df_cln.FORMATION=='0', 'FORMATION']='None'
#Save data to parquet
# df_cln.to_parquet('ACG_wells_JOINT_BEST_v7.parquet.gzip', compression='gzip')

In [3]:
#Loading metadata, distribution wells per Platforms and all the that.
path = 'C:\\jupyter\\SPP\\input\\'
metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
metadata = metadata_init.copy()
metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
metadata.Status = metadata.Status.str.strip()
metadata.Status = metadata.Status.str.lower()
metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

#Read data from parquet
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
df_prq.rename(columns={'wellName':'well'}, inplace=True)
df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
# print('wells in df totally:', len(df_prq.well.unique()))
# Filter data with bad_well_list 
bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP']
df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
#Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
#Getting XY coords of Balakhany formation tops
xy_coord = df_prq[['well', 'FORMATION', 'X', 'Y']]
xy_coord = xy_coord.groupby(['well', 'FORMATION']).apply(lambda x: x.iloc[0]).drop(columns=['well', 'FORMATION']).reset_index()
xy_coord = xy_coord[xy_coord.FORMATION.str.contains('Balakhany') & (xy_coord.X>0) & (xy_coord.Y>0)]
#Find top TVD_SCS for each formation
df_prq_tvdss = df_prq[['well','MD','FORMATION','TVD_SCS']].groupby(['well','FORMATION']).apply(lambda x: x.iloc[0])
df_prq_tvdss = df_prq_tvdss.drop(['well','FORMATION'], axis=1).reset_index()
df_prq_tvdss = df_prq_tvdss[df_prq_tvdss.TVD_SCS>0]

## Data preparation

### Cleaning dataset for outliers

In [5]:
#Reading csv with initial KHtst_v3, joining xy-coord & TVD_SCS tops of formation
path = 'C:\\jupyter\\SPP\\inputoutput\\'
df_khtst = pd.read_csv(path + 'df_prq_khtst_v3.csv')
df_khtst_xy = df_khtst.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_khtst_xy_tvd = df_khtst_xy.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_khtst_xy_tvd_fld = df_khtst_xy_tvd.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
df_khtst_xy_tvd_fld = df_khtst_xy_tvd_fld[df_khtst_xy_tvd_fld.TVD_SCS.notna()]
#Clean dataset for outliers for Balakhany VIII & X  for AZR and CHG fields by rule 1.5 * IQR
fm_list_8_10 = ['Balakhany VIII', 'Balakhany VIII sand', 'Balakhany VIII 25','Balakhany VIII 20', 
                'Balakhany VIII 15', 'Balakhany VIII 10', 'Balakhany VIII 5',
                'Balakhany X', 'Balakhany X sand', 'Balakhany X 40', 'Balakhany X 20'] 
azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI']
chg_lst = ['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG']
df_lst = []
for fm in fm_list_8_10:
    df_khtst_fm = df_khtst_xy_tvd_fld[(df_khtst_xy_tvd_fld.FORMATION == fm) & (df_khtst_xy_tvd_fld.field.isin(azr_lst))]
    Q1 = df_khtst_fm['KHtst'].quantile(0.25)
    Q3 = df_khtst_fm['KHtst'].quantile(0.75)
    IQR = Q3 - Q1
    # print(f'bal {fm} azr IQR', IQR, 'bot limit:', (Q1 - 1.5 * IQR), 'top limit:', (Q3 + 1.5 * IQR))
    df_khtst_fm_qcl = df_khtst_fm[~((df_khtst_fm['KHtst'] < (Q1 - 1.5 * IQR)) | (df_khtst_fm['KHtst'] > (Q3 + 1.5 * IQR)))]
    df_lst.append(df_khtst_fm_qcl)
for fm in fm_list_8_10:
    df_khtst_fm = df_khtst_xy_tvd_fld[(df_khtst_xy_tvd_fld.FORMATION == fm) & (df_khtst_xy_tvd_fld.field.isin(chg_lst))]
    Q1 = df_khtst_fm['KHtst'].quantile(0.25)
    Q3 = df_khtst_fm['KHtst'].quantile(0.75)
    IQR = Q3 - Q1
    # print(f'bal {fm} chg IQR', IQR, 'bot limit:', (Q1 - 1.5 * IQR), 'top limit:', (Q3 + 1.5 * IQR))
    df_khtst_fm_qcl = df_khtst_fm[~((df_khtst_fm['KHtst'] < (Q1 - 1.5 * IQR)) | (df_khtst_fm['KHtst'] > (Q3 + 1.5 * IQR)))]
    df_lst.append(df_khtst_fm_qcl)
df_khtst_bal_qcl = pd.concat(df_lst)

### List of functions

In [6]:
#Uploading k_htst data from csv-file & Calculation of Euclidean Distances
path = 'C:\\jupyter\\SPP\\inputoutput\\'
df_khtst = pd.read_csv(path + 'df_prq_khtst_v3.csv')
df_khtst_xy = df_khtst.set_index(['well','FORMATION']).join(
                                                            df_prq[['well','FORMATION','X','Y','TVD_SCS']].groupby(
                                                            ['well','FORMATION']).apply(lambda x: x.iloc[0]).drop(
                                                            ['well','FORMATION'], axis=1)
                                                            ).reset_index()
# Function to display well plots with logging curves
def well_display(wellname, fmname):
    data = df_prq[(df_prq.well==wellname) & (df_prq.FORMATION == fmname)]
    depth = data['MD']
    grn = data['GR_N']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    fluid = data['gross_pay']
    perm = data['LPERM']
    fig, ax = plt.subplots(1,5, figsize=(7,7), sharey=True)
    ax[0].plot(grn, depth, color='lightgreen'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150)
    ax[1].plot(rhob, depth, color='red'), ax[1].invert_yaxis(), ax[1].set_xlim(1.65, 2.65)
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue')
    twin1.set_xlim(0.6, 0)
    ax[2].plot(rdeep, depth, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(1, 100), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[3].plot(phit, depth, color='green'), ax[3].set_xlim(0.3, 0), ax[3].grid(axis='x'), ax[3].invert_yaxis()
    ax[3].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[3].twiny()
    twin2.plot(fluid, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,fluid, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    ax[4].plot(perm, depth, color='purple'), ax[4].set_xscale('log'), ax[4].set_xlim(1, 1000), ax[4].grid(axis='x'), ax[4].invert_yaxis()
    fig.suptitle(wellname + ' ' + fmname, fontsize=14)
    fig.tight_layout()
#Calculation of Euclidean Distances for the top of Balakhany VIII sand & Balakhany X sand
def well_dist_calc(dist_formation, iqr_clean, iqr_wells):
    """
    dist_formation, iqr_clean=f(1,0), iqr_wells == df_khtst_bal_qcl
    """
    data = df_khtst_xy_tvd_fld[(df_khtst_xy_tvd_fld.FORMATION == dist_formation) & (df_khtst_xy_tvd_fld.X > 0) & (df_khtst_xy_tvd_fld.Y > 0)]
    if iqr_clean == 1:
        data = data[data.well.isin(iqr_wells)]
    else:
        pass
    row_name = data.well.reset_index().drop(['index'], axis=1)
    distance_fm = pd.DataFrame(euclidean_distances(data[['X', 'Y', 'TVD_SCS']]), columns=list(data.well))
    distance_fm_well = distance_fm.join(row_name).set_index('well')
    distance_fm_well.reset_index()
    return distance_fm_well.reset_index()
#Collecting of EucDist, XY and KHtst data 
def well_kh_accum(data, kh_formation):
    well_kh_accum = []
    well_x_accum = []
    well_y_accum = []
    for i in data:
        well_kh_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['KHtst'].reset_index())    
        well_x_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['X'].reset_index())
        well_y_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['Y'].reset_index())
    well_kh3 = pd.concat(well_kh_accum).T[1:]
    well_kh3.columns = ['kh1','kh2','kh3']
    well_x3 = pd.concat(well_x_accum).T[1:]
    well_x3.columns = ['x1','x2','x3']
    well_y3 = pd.concat(well_y_accum).T[1:]
    well_y3.columns = ['y1','y2','y3']
    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                        well_x3.reset_index().drop('index',axis=1), 
                        well_y3.reset_index().drop('index',axis=1)], axis=1)
    return final
#Collecting of Dist & Wells data togher with func(well_kh_accum)
def well_offsets_collection(dist_dataset, off_formation, kh_dataset, metadata):
    df_collect = []
    for num, well_name in enumerate(dist_dataset.well[:]):
        well_dist3 = dist_dataset[dist_dataset.well == well_name].T[1:].sort_values(by=num)[1:4].reset_index()
        well_dist3_tuple = tuple(well_dist3['index'])
        well_dist3_res = well_dist3.T[1:].reset_index().drop('index', axis=1)   
        well_name3_res = well_dist3.T[:1].reset_index().drop('index', axis=1)
        well_kh3_res = well_kh_accum(well_dist3_tuple, off_formation)
        well_dist3_res.columns =['dist1', 'dist2', 'dist3']
        well_name3_res.columns =['well1', 'well2', 'well3']
        concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
        result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
        df_collect.append(result)       
    df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
    df_khtst_xy_bal = kh_dataset[kh_dataset.FORMATION==off_formation][['well', 'FORMATION', 'KHtst']]
    df_well_kh_dist_frm = df_well_kh_dist.set_index('well').join(df_khtst_xy_bal.set_index('well')).reset_index()
    df_well_kh_dist_frm_fld = df_well_kh_dist_frm.set_index('well').join(metadata[['well','field']].set_index('well')).reset_index()
    return df_well_kh_dist_frm_fld
#Calculating avg properties and collecting df per FU
def df_avg_properties(formation_avg_prop, df_avg_prop, df_joined):
    df_hpv = df_joined[df_joined.FORMATION == formation_avg_prop][['well','FORMATION','res_tst','phit_wavg','vsh_wavg']]
    df_permh = df_avg_prop[df_avg_prop.FORMATION == formation_avg_prop].groupby(['well','FORMATION'])['kavg_htst'].sum().reset_index()
    df_phhpv = df_hpv.set_index(['well','FORMATION']).join(df_permh.set_index(['well','FORMATION'])).reset_index()
    return df_phhpv
#Rotate x,y around xo,yo by theta (rad)
def rotate(x,y): 
    theta = (math.pi/180)*34
    xo = st.median(np.array(df_khtst_xy['X'].to_list()))
    yo = st.median(np.array(df_khtst_xy['Y'].to_list()))
    xr = math.cos(theta)*(x-xo)-math.sin(theta)*(y-yo) + xo
    yr = math.sin(theta)*(x-xo)+math.cos(theta)*(y-yo) + yo
    return [xr,yr]

### Data collection

In [7]:
# Selecting wells without outliers
well_clean_8a = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII')].well 
well_clean_8 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII sand')].well
well_clean_10a = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X')].well
well_clean_10 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X sand')].well
well_clean_10_20 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X 20')].well
well_clean_10_40 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X 40')].well
well_clean_8_25 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 25')].well
well_clean_8_20 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 20')].well
well_clean_8_15 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 15')].well
well_clean_8_10 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 10')].well
well_clean_8_5 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 5')].well
# Distance calculation
qc_n = 1
dist_bal8a = well_dist_calc('Balakhany VIII', qc_n, well_clean_8a)
dist_bal8 = well_dist_calc('Balakhany VIII sand', qc_n, well_clean_8)
dist_bal10a = well_dist_calc('Balakhany X', qc_n, well_clean_10a)
dist_bal10 = well_dist_calc('Balakhany X sand', qc_n, well_clean_10)
dist_bal10_20 = well_dist_calc('Balakhany X 20', qc_n, well_clean_10_20)
dist_bal10_40 = well_dist_calc('Balakhany X 40', qc_n, well_clean_10_40)
dist_bal8_25 = well_dist_calc('Balakhany VIII 25', qc_n, well_clean_8_25)
dist_bal8_20 = well_dist_calc('Balakhany VIII 20', qc_n, well_clean_8_20)
dist_bal8_15 = well_dist_calc('Balakhany VIII 15', qc_n, well_clean_8_15)
dist_bal8_10 = well_dist_calc('Balakhany VIII 10', qc_n, well_clean_8_10)
dist_bal8_5 = well_dist_calc('Balakhany VIII 5', qc_n, well_clean_8_5)
# Creating datasets according to FU
df_well_kh_dist_bal8a_fld = well_offsets_collection(dist_bal8a, 'Balakhany VIII', df_khtst_xy, metadata)
df_well_kh_dist_bal8_fld = well_offsets_collection(dist_bal8, 'Balakhany VIII sand', df_khtst_xy, metadata)
df_well_kh_dist_bal8_25_fld = well_offsets_collection(dist_bal8_25, 'Balakhany VIII 25', df_khtst_xy, metadata)
df_well_kh_dist_bal8_20_fld = well_offsets_collection(dist_bal8_20, 'Balakhany VIII 20', df_khtst_xy, metadata)
df_well_kh_dist_bal8_15_fld = well_offsets_collection(dist_bal8_15, 'Balakhany VIII 15', df_khtst_xy, metadata)
df_well_kh_dist_bal8_10_fld = well_offsets_collection(dist_bal8_10, 'Balakhany VIII 10', df_khtst_xy, metadata)
df_well_kh_dist_bal8_5_fld = well_offsets_collection(dist_bal8_5, 'Balakhany VIII 5', df_khtst_xy, metadata)
df_well_kh_dist_bal10a_fld = well_offsets_collection(dist_bal10a, 'Balakhany X', df_khtst_xy, metadata)                                   
df_well_kh_dist_bal10_fld = well_offsets_collection(dist_bal10, 'Balakhany X sand', df_khtst_xy, metadata)
df_well_kh_dist_bal10_40_fld = well_offsets_collection(dist_bal10_40, 'Balakhany X 40', df_khtst_xy, metadata)
df_well_kh_dist_bal10_20_fld = well_offsets_collection(dist_bal10_20, 'Balakhany X 20', df_khtst_xy, metadata)

In [ ]:
#Concatenation all dist-datasets                                                  
df_well_kh_dist_all = pd.concat([   df_well_kh_dist_bal8a_fld,
                                    df_well_kh_dist_bal8_fld,    
                                    df_well_kh_dist_bal8_25_fld,
                                    df_well_kh_dist_bal8_20_fld,
                                    df_well_kh_dist_bal8_15_fld,
                                    df_well_kh_dist_bal8_10_fld, 
                                    df_well_kh_dist_bal8_5_fld,
                                    df_well_kh_dist_bal10a_fld,                                    
                                    df_well_kh_dist_bal10_fld,
                                    df_well_kh_dist_bal10_40_fld,
                                    df_well_kh_dist_bal10_20_fld])
df_well_kh_dist_all.shape

In [9]:
#Calculation of TST-thickness Balakhany VIII & X
df_fu_tst = df_prq[(df_prq.FORMATION.str.contains('Balakhany VIII')) | (df_prq.FORMATION.str.contains('Balakhany X'))]
df_fu_tst = df_fu_tst[['well', 'MD','FORMATION','TST']]
df_fu_tst_top = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[0]).reset_index()
df_fu_tst_top.rename(columns={'TST':'TST_top'}, inplace=True)
df_fu_tst_bot = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1]).reset_index()
df_fu_tst_bot.rename(columns={'TST':'TST_bot'}, inplace=True)
df_fu_tst_final = df_fu_tst_top.set_index(['well','FORMATION']).join(df_fu_tst_bot.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final['TST_interv'] = round((df_fu_tst_final.TST_bot - df_fu_tst_final.TST_top),0)
#Joining coordinates datasets
df_fu_tst_final = df_fu_tst_final.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
df_fu_tst_final = df_fu_tst_final[(df_fu_tst_final.TST_interv > 0)]

In [11]:
#Reading df_prq_htst_avgprop_v1 and getting outliers
path = 'C:\\jupyter\\SPP\\inputoutput\\' 
df_htst_avgprop = pd.read_csv(path + 'df_prq_htst_avgprop_v1.csv')
#Preparation weighted average df_htst_avgprop-dataset
cutoff_h_tst = -1
cutoff_perm_avg = -1
#Applying filtration to dataset with cutoffs
df_htst_avgprop_nz = df_htst_avgprop[(df_htst_avgprop.h_tst > cutoff_h_tst) & (df_htst_avgprop.md_perm_avg > cutoff_perm_avg)]
#Multiplaying htst by resprop values
df_htst_avgprop_nz['kavg_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_perm_avg
df_htst_avgprop_nz['phit_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_phit_avg
df_htst_avgprop_nz['vsh_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_vsh_avg
#Summarizing h_tst via well & formation
df_htst_fm = df_htst_avgprop_nz.groupby(['well','FORMATION'])['h_tst'].sum().reset_index()
df_htst_fm.rename(columns={'h_tst':'res_tst'}, inplace=True)
#Calculating weighted averages
df_htst_avgprop_nz_avgpropsum = df_htst_avgprop_nz.groupby(['well','FORMATION'])[['phit_htst','vsh_htst']].sum().reset_index()
df_htst_avgprop_nz_avgpropsum_join = df_htst_avgprop_nz_avgpropsum.set_index(
                                     ['well','FORMATION']).join(df_htst_fm.set_index(['well','FORMATION'])).reset_index()
df_htst_avgprop_nz_avgpropsum_join['phit_wavg'] = df_htst_avgprop_nz_avgpropsum_join.phit_htst / df_htst_avgprop_nz_avgpropsum_join.res_tst
df_htst_avgprop_nz_avgpropsum_join['vsh_wavg'] = df_htst_avgprop_nz_avgpropsum_join.vsh_htst / df_htst_avgprop_nz_avgpropsum_join.res_tst

In [ ]:
#Calculating avg properties and collecting df per FU
df_8abal_phhpv = df_avg_properties('Balakhany VIII', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_phhpv = df_avg_properties('Balakhany VIII sand', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10abal_phhpv = df_avg_properties('Balakhany X', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_phhpv = df_avg_properties('Balakhany X sand', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_20_phhpv = df_avg_properties('Balakhany X 20', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_40_phhpv = df_avg_properties('Balakhany X 40', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_25_phhpv = df_avg_properties('Balakhany VIII 25', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_20_phhpv = df_avg_properties('Balakhany VIII 20', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_15_phhpv = df_avg_properties('Balakhany VIII 15', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_10_phhpv = df_avg_properties('Balakhany VIII 10', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_5_phhpv = df_avg_properties('Balakhany VIII 5', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
#Concatenation of all df per FU
df_all_bal_phhpv = pd.concat([  df_8abal_phhpv, df_8bal_phhpv, df_10abal_phhpv, df_10bal_phhpv, df_10bal_20_phhpv, df_10bal_40_phhpv, 
                                df_8bal_25_phhpv, df_8bal_20_phhpv, df_8bal_15_phhpv, df_8bal_10_phhpv, df_8bal_5_phhpv])
df_all_bal_phhpv_tstint = df_all_bal_phhpv.set_index(['well','FORMATION']).join(df_fu_tst_final.set_index(['well','FORMATION'])).reset_index()
df_avgprop_final_wa = df_all_bal_phhpv_tstint.copy()
df_avgprop_final_wa.shape

In [13]:
# For Shahriyar
perm_cutoff = -1
perm_cutoff_top = 140_000
df_dist_kh_bal_shahriayr_init = df_avgprop_final_wa.set_index(['well','FORMATION']).join(
                                df_well_kh_dist_all.drop('field',axis=1).set_index(['well','FORMATION'])).reset_index()
df_dist_kh_bal_shahriayr_init.rename(columns={'TST_interv':'interv_tst', 'TST_top':'tst_top', 'TST_bot':'tst_bot'}, inplace=True)
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr_init[df_dist_kh_bal_shahriayr_init.MD.notna()]
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr[ (df_dist_kh_bal_shahriayr.kh1>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kh2>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kh3>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kavg_htst>perm_cutoff)]
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr[ (df_dist_kh_bal_shahriayr.kh1<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kh2<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kh3<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kavg_htst<perm_cutoff_top)]
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
df_dist_kh_bal_shahriayr['kavg_htst_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kavg_htst'])
df_dist_kh_bal_shahriayr['kh1_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh1'])
df_dist_kh_bal_shahriayr['kh2_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh2'])
df_dist_kh_bal_shahriayr['kh3_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh3'])
df_dist_kh_bal_shahriayr[['X_new', 'Y_new']] = df_dist_kh_bal_shahriayr.apply(lambda row: rotate(row['X'], row['Y']), axis=1, result_type='expand')
df_dist_kh_bal_shahriayr = pd.get_dummies(df_dist_kh_bal_shahriayr, columns = ['FORMATION_up'])
# df_dist_kh_bal_shahriayr_init.info()

## Maps building

### Summarize all FU tst_interv for Bal 8 & Bal 10

In [ ]:
# Distribution tst-thickness Balaknany VIII / X over Chirag and Azeri zones
# Calculation of TST-thickness Balakhany VIII & X
def histo_creation(data_ds1,ds1_label, data_ds2, ds2_label, title_ds):
    fig = go.Figure()
    fig.add_trace(go.Histogram(
                x=data_ds1.TST_interv, 
                xbins=dict(start=0, end=300 , size=5), marker_color='yellow', name=ds1_label))
    fig.add_trace(go.Histogram(
                x=data_ds2.TST_interv, 
                xbins=dict(start=0, end=300, size=5), marker_color='blue', name=ds2_label))
    fig.update_traces(opacity=0.75)
    fig.update_layout(title_text=title_ds,
                    xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                    autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
    fig.update_layout(barmode='stack')
    fig.update_xaxes(nticks=40, showgrid=True)
    return fig.show()
df_fu_tst = df_prq[(df_prq.FORMATION.str.contains('Balakhany VIII')) | (df_prq.FORMATION.str.contains('Balakhany X'))]
df_fu_tst = df_fu_tst[['well', 'MD','FORMATION','TST']]
df_fu_tst_top = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[0]).reset_index()
df_fu_tst_top.rename(columns={'TST':'TST_top'}, inplace=True)
df_fu_tst_bot = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1]).reset_index()
df_fu_tst_bot.rename(columns={'TST':'TST_bot'}, inplace=True)
df_fu_tst_final = df_fu_tst_top.set_index(['well','FORMATION']).join(df_fu_tst_bot.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final['TST_interv'] = round((df_fu_tst_final.TST_bot - df_fu_tst_final.TST_top),0)
df_fu_tst_final = df_fu_tst_final.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
df_fu_tst_final = df_fu_tst_final[(df_fu_tst_final.TST_interv > 0)]
df_fu_tst_final.loc[df_fu_tst_final.field == 'DDGG', 'field_num'] = 1
df_fu_tst_final.loc[df_fu_tst_final.field == 'DWG', 'field_num'] = 2
df_fu_tst_final.loc[df_fu_tst_final.field == 'WEST CHIRAG', 'field_num'] = 3
df_fu_tst_final.loc[df_fu_tst_final.field == 'CHIRAG', 'field_num'] = 4
df_fu_tst_final.loc[df_fu_tst_final.field == 'WEST AZERI', 'field_num'] = 5
df_fu_tst_final.loc[df_fu_tst_final.field == 'CENTRAL AZERI', 'field_num'] = 6
df_fu_tst_final.loc[df_fu_tst_final.field == 'EAST AZERI', 'field_num'] = 7
# Summarize all FU tst_interv for Bal 8 & Bal 10
df_fu_tst_final_8 = df_fu_tst_final[df_fu_tst_final.FORMATION.str.contains('Balakhany VIII')]
df_fu_tst_final_10 = df_fu_tst_final[df_fu_tst_final.FORMATION.str.contains('Balakhany X')]
df_fu_tst_final_10_tstinterv = df_fu_tst_final_10.groupby('well')['TST_interv'].sum().reset_index()
df_fu_tst_final_8_tstinterv = df_fu_tst_final_8.groupby('well')['TST_interv'].sum().reset_index()
df_8_tstinterv = df_fu_tst_final_8_tstinterv.set_index('well').join(df_fu_tst_final_8[['well','field_num']].set_index('well'))
df_10_tstinterv = df_fu_tst_final_10_tstinterv.set_index('well').join(df_fu_tst_final_10[['well','field_num']].set_index('well'))
# Distribution tst-thickness Balaknany VIII / X over Chirag and Azeri zones
df_8_tstinterv_chi = df_8_tstinterv[df_8_tstinterv.field_num <= 4]
df_8_tstinterv_azr = df_8_tstinterv[df_8_tstinterv.field_num > 4]
df_10_tstinterv_chi = df_10_tstinterv[df_10_tstinterv.field_num <= 4]
df_10_tstinterv_azr = df_10_tstinterv[df_10_tstinterv.field_num > 4]
# Histograms per Bal8 & Bal10
histo_creation(df_8_tstinterv_chi, 'bal8_crg', df_8_tstinterv_azr, 'bal8_azr', 'Bal 8 chi & azr')
histo_creation(df_10_tstinterv_chi, 'bal10_crg', df_10_tstinterv_azr, 'bal10_azr','Bal 10 chi & azr')

### Building map KHtst for whole Bal_8 and Bal_10 intervals with summarization of all subsidiary FU

In [ ]:
#Reading df_avgprop_final_wa and getting outliers
df_summ_bal810 = df_avgprop_final_wa.copy()
df_summ_bal810.loc[df_summ_bal810.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
df_summ_bal810.loc[df_summ_bal810.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
df_bal8_10 = df_summ_bal810.copy()
df_bal8_10_gb = df_bal8_10.groupby(['well','FORMATION_up'])['kavg_htst'].sum().reset_index()
df_bal8_10_gb_xy = df_bal8_10.groupby(['well','FORMATION_up'])[['X','Y','TVD_SCS']].mean().reset_index()
df_bal8_10_fin = df_bal8_10_gb.set_index(['well', 'FORMATION_up']).join(df_bal8_10_gb_xy.set_index(['well', 'FORMATION_up'])).reset_index()
df_bal8_10_fin_fld = df_bal8_10_fin.set_index(['well']).join(df_bal8_10[['well','field']].set_index(['well']), how='inner').reset_index()
df_bal8_10_fin_fld.drop_duplicates(subset=['X','Y','field'], keep='first', inplace=True)
df_bal8_10_fin_fld.rename(columns={'kavg_htst':'kavg_htst_up'}, inplace=True)
df_bal8_10_fin_fld['kavg_htst_up'] = df_bal8_10_fin_fld.kavg_htst_up.round(0)
df_bal8_10_fin_fld

In [16]:
# gdf = gpd.read_file("input\polygons_geojson\E10_tbal_Poor_Data_Quality_ZARF_5174_M400000_QLSKPrSDM_meaefx.ptd.geojson")
# gdf.plot(aspect='equal', figsize=(12,8))
# plt.grid()
# plt.scatter(df_bal8_10_fin_fld.X, df_bal8_10_fin_fld.Y, c='black')
# plt.show()

In [ ]:
def map_kh_2plots(metadata, kh_dataset, formation, kh_value, multi_chr = 0.002, multi_azr = 0.001):
    """
    metadata, 
    kh_dataset = df_khtst_xy_tvd_fld, 
    formation = 'Balakhany VIII',  
    kh_value = 'KHtst',
    multi_chr = 0.001, multi_azr = 0.001
    """
    fig = make_subplots(rows=2, cols=1, subplot_titles=("CHG, 0.0025", "AZR, 0.0075"), 
                        vertical_spacing = 0.025)
    azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI']
    chg_lst = ['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    field_avg_coord_chg = field_avg_coord[field_avg_coord.field.isin(chg_lst)]
    field_avg_coord_azr = field_avg_coord[field_avg_coord.field.isin(azr_lst)] 
    df_chg = kh_dataset[(kh_dataset.FORMATION == formation) & (kh_dataset.field.isin(chg_lst))]
    df_azr = kh_dataset[(kh_dataset.FORMATION == formation) & (kh_dataset.field.isin(azr_lst))]
    # df_khtst_xy_10high = df_khtst_xy_tvd[(df_khtst_xy_tvd.FORMATION == 'Balakhany X sand') & (df_khtst_xy_tvd.well.isin(khtst_map_hwells))]
    fig.add_trace(go.Scatter(x=df_chg.X, y=df_chg.Y, customdata = df_chg[['well', kh_value]],
                            marker=dict(color=df_chg.TVD_SCS, size=df_chg[kh_value]*multi_chr, colorscale='Viridis_r',  showscale=True,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', name='KHtst chirag', hovertemplate="".join(["well:%{customdata[0]}, KHtst:%{customdata[1]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_chg.X_wellhead, y=field_avg_coord_chg.Y_wellhead, customdata = field_avg_coord_chg[['field']],
                            text=field_avg_coord_chg['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', name='Platforms', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=df_azr.X, y=df_azr.Y, customdata = df_azr[['well', kh_value]],
                            marker=dict(color=df_azr.TVD_SCS, size=df_azr[kh_value]*multi_azr, colorscale='Viridis_r',  showscale=False,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', name='KHtst azeri', hovertemplate="".join(["well:%{customdata[0]}, KHtst:%{customdata[1]}<extra></extra>"])),
                            row=2, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_azr.X_wellhead, y=field_avg_coord_azr.Y_wellhead, customdata = field_avg_coord_azr[['field']],
                            text=field_avg_coord_azr['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', name='Platforms', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=2, col=1)
    fig.update_layout(title_text= formation + 'size=f(KHtst), color=f(TVD_SCS), cleaned by rule 1.5*IQR',
                    autosize=True, width=1300, height=1400, margin=dict(l=10,r=10,b=10,t=50))
    fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
    return fig.show()
map_kh_2plots(metadata, df_khtst_xy_tvd_fld, 'Balakhany VIII', 'KHtst', 0.02, 0.01)

In [ ]:
df_khtst_xy_tvd_fld.head(3)

In [20]:
# formation = 'Balakhany VIII'
# for well in df_khtst_xy_tvd_fld[df_khtst_xy_tvd_fld.FORMATION == formation].well.unique()[:5]:
#     well_display(well, formation)

### Well plot verifikation

In [1]:
# len(df_prq[((df_prq.FORMATION == fmname) & (df_prq.PHIT>0.13))].well.unique())

In [ ]:
fmname = 'Balakhany VIII'
for w in df_prq[(df_prq.FORMATION == fmname)].well.unique():
    well_display(w, fmname)